### 基本使用示例

In [2]:
_functions = {}
def register(f):
    global _functions
    _functions[f.__name__] = f
    return f

@register
def foo():
    return 'bar'

In [3]:
_functions

{'foo': <function __main__.foo>}

In [4]:
foo()

'bar'

### 实际例子

In [22]:
# 1.原始代码
class Store(object):
    def get_food(self, username, food):
        if username != 'admin':
            raise Exception("This user is not allowed to get food")
        return self.storage.get(food)
    
    def get_food(self, username, food):
        if username != 'admin':
            raise Exception("This user is not allowed to put food")
        return self.storage.put(food)
    
# 2.拆分成函数
def check_is_admin(username):
    if username != 'admin':
        raise Exception("This user is not allowed to get food")
    
class Store(object):
    def get_food(self, username, food):
        check_is_admin(username)
        return self.storage.get(food)
    
    def get_food(self, username, food):
        check_is_admin(username)
        return self.storage.put(food)

# 3.使用装饰器
def check_is_admin(f):
    def wrapper(*args, **kwargs):
        if kwargs.get('username') != 'admin':
            raise Exception("This user is not allowed to get food")
        return f(*args, **kwargs)
    return wrapper
    
class Store(object):
    @check_is_admin
    def get_food(self, username, food):
        return self.storage.get(food)
    
    @check_is_admin
    def get_food(self, username, food):
        return self.storage.put(food)

In [27]:
def function0(username, food):
    username += '0'
    return username,food
    
w = check_is_admin(function0)
w(username = 'admin',food = 'd')

('admin0', 'd')

#### 匿名参数 arg,*args,**kwargs

In [5]:
def function1(arg,*args,**kwargs):
    print(arg,args,kwargs)

function(6,7,8,9,a=1, b=2, c=3)

6 (7, 8, 9) {'b': 2, 'a': 1, 'c': 3}


In [10]:
def function2(*args,**kwargs):
    print(args,kwargs)

function(username ='admin', food ='apple')

() {'username': 'admin', 'food': 'apple'}


### 进阶

装饰器会用一个动态创建的新函数替换原函数，但新函数缺少很多原函数的属性，如docstring和名字。

In [32]:
def check_is_admin(f):
    def wrapper(*args, **kwargs):
        if kwargs.get('username') != 'admin':
            raise Exception("This user is not allowed to get food")
        return f(*args, **kwargs)
    return wrapper

def foobar(username = 'someone'):
    """Do crazy stuff."""
    pass

print(foobar.__doc__, '\n', foobar.__name__)

@check_is_admin
def foobar2(username = 'someone'):
    """Do crazy stuff."""
    pass

print(foobar2.__doc__, '\n', foobar2.__name__)

Do crazy stuff. 
 foobar
None 
 wrapper


通过内置的functools模块update_wrapper函数可以解决这个问题，它会复制这些属性给这个包装器

In [34]:
import functools
def foobar3(username = 'someone'):
    """Do crazy stuff."""
    pass
foobar3 = functools.update_wrapper(check_is_admin, foobar3)
print(foobar3.__doc__, '\n', foobar3.__name__)

Do crazy stuff. 
 foobar3


手工调用update_wrapper创建装饰器很不方便，所以functools提供了名为wraps的装饰器:

In [35]:
def check_is_admin(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        if kwargs.get('username') != 'admin':
            raise Exception("This user is not allowed to get food")
        return f(*args, **kwargs)
    return wrapper
    
class Store(object):
    @check_is_admin
    def get_food(self, username, food):
        return self.storage.get(food)
    

目前为止，在我们的示例中总是假设被装饰的函数会有一个名为username的关键字参数传入，但情况并非总是如此。考虑到这一点，最好提供一个更加智能的装饰器，它能查看被装饰函数的参数并从中提取需要的参数，

为此，inspect模块允许提取函数的签名并对其进行操作。

In [41]:
import inspect

def check_is_admin(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        func_args = inspect.getcallargs(f, *args, **kwargs)
        # inspect.getcallargs 返回一个将参数名字和值作为键值对的字典 意味着装饰器不用检查是基于位置的参数还是关键字参数
        if func_args.get('username') != 'admin':
            raise Exception("This user is not allowed to get food")
        return f(*args, **kwargs)
    return wrapper
    

@check_is_admin
def get_food(time,username, type = 'chocolate'):
    return type + " nom nom nom!"

get_food('today',username = 'admin')

'chocolate nom nom nom!'

> 参考：Python高手之路